<a href="https://colab.research.google.com/github/luizthompson/MVP_Sprint_Netflix/blob/main/MVP_Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP - Sprint: Machine Learning & Analytics
Luiz Guilherme Thompson Vaz

# Definição do Problema

Descrição do Problema:
O problema que estamos tentando resolver é a previsão do preço de fechamento do dia seguinte das ações da Netflix. Isso é um problema de regressão no campo de aprendizado de máquina.

# Premissas ou Hipóteses

A hipótese fundamental é que os preços das ações são influenciados por uma combinação de fatores técnicos (como preços de abertura, máximo, mínimo e fechamento de cada dia de negociação, volume de negociação, indicadores de impulso, etc.) e que esses fatores podem ser usados para prever movimentos futuros de preços.

# Restrições ou Condições para Selecionar os Dados

Os dados utilizados para este projeto são históricos e foram coletados ao longo da última década. Eles incluem uma variedade de indicadores técnicos e estatísticas vitais. No entanto, eles não incluem informações sobre eventos do mundo real (como notícias ou anúncios da empresa) que também podem influenciar os preços das ações.

## Descrição do Dataset

O conjunto de dados inclui a data de registro e diversas estatísticas vitais: os preços de abertura, máximo, mínimo e fechamento de cada dia de negociação, juntamente com o volume de negociação. Ele também contém indicadores de impulso, como o Índice de Força Relativa (RSI) de 7 e 14 dias, o Commodity Channel Index (CCI) para 7 e 14 dias, a média móvel simples (SMA) de 50 e 100 dias e a média móvel exponencial (EMA). Indicadores adicionais importantes são a Convergência Divergência das Médias Móveis (MACD), as Bandas de Bollinger, o True Range e o Average True Range (ATR) de 7 e 14 dias. O objetivo do conjunto de dados é prever o preço de fechamento do dia seguinte das ações da Netflix.

In [67]:
# Importando as bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Carregar o dataset
url = 'https://raw.githubusercontent.com/luizthompson/MVP_Sprint_Netflix/main/nflx_2014_2023.csv'
df = pd.read_csv(url)

# Exibindo as primeiras linhas do dataset
print(df.head())

         date       open       high        low      close    volume  \
0  2014-01-02  52.401428  52.511429  51.542858  51.831429  12325600   
1  2014-01-03  52.000000  52.495712  51.842857  51.871429  10817100   
2  2014-01-06  51.889999  52.044285  50.475716  51.367142  15501500   
3  2014-01-07  49.684284  49.698570  48.152859  48.500000  36167600   
4  2014-01-08  48.104286  49.425713  48.074287  48.712856  20001100   

       rsi_7     rsi_14       cci_7      cci_14     sma_50     ema_50  \
0  34.729664  49.183584  -89.573201 -131.288579  50.112828  50.235157   
1  35.587886  49.457208  -65.820581 -103.026189  50.228771  50.299327   
2  29.820674  46.087900 -121.472559 -139.640566  50.312571  50.341203   
3  14.371863  32.522091 -206.762171 -238.029120  50.336228  50.268997   
4  18.049045  34.073549 -117.836707 -180.766801  50.373257  50.207969   

     sma_100    ema_100      macd  bollinger  TrueRange     atr_7    atr_14  \
0  46.385428  46.650698  0.751929  52.607357   1.052857

In [68]:
# Vou verificar as informações gerais do meu dataframe (tipos de dados, número de valores não nulos, etc.)
print(df.info())

# Vou verificar as estatísticas descritivas do meu dataframe (média, desvio padrão, valores mínimos/máximos, etc.)
print(df.describe())

# Vou verificar se há valores faltantes no meu dataframe
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2516 entries, 0 to 2515
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            2516 non-null   object 
 1   open            2516 non-null   float64
 2   high            2516 non-null   float64
 3   low             2516 non-null   float64
 4   close           2516 non-null   float64
 5   volume          2516 non-null   int64  
 6   rsi_7           2516 non-null   float64
 7   rsi_14          2516 non-null   float64
 8   cci_7           2516 non-null   float64
 9   cci_14          2516 non-null   float64
 10  sma_50          2516 non-null   float64
 11  ema_50          2516 non-null   float64
 12  sma_100         2516 non-null   float64
 13  ema_100         2516 non-null   float64
 14  macd            2516 non-null   float64
 15  bollinger       2516 non-null   float64
 16  TrueRange       2516 non-null   float64
 17  atr_7           2516 non-null   f

# Preparação de Dados

Separação do Dataset
Vou dividir o conjunto de dados em um conjunto de treinamento e um conjunto de teste. O conjunto de treinamento é usado para treinar o modelo, enquanto o conjunto de teste é usado para avaliar o desempenho do modelo.

In [69]:
from sklearn.model_selection import train_test_split

# Definir a variável alvo
target = df['next_day_close']

# Definir as variáveis independentes
features = df.drop('next_day_close', axis=1)

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Verificar o tamanho dos conjuntos de treino e teste
print("Tamanho do conjunto de treino:", X_train.shape)
print("Tamanho do conjunto de teste:", X_test.shape)

Tamanho do conjunto de treino: (2012, 19)
Tamanho do conjunto de teste: (504, 19)


# Validação Cruzada
Faz sentido usar a validação cruzada. A validação cruzada é uma técnica poderosa que fornece uma estimativa mais robusta do desempenho do modelo. Ela reduz a variância associada à divisão única de treino/teste, garantindo que o modelo seja testado em diferentes subconjuntos do conjunto de dados. No entanto, ela pode ser mais demorada, pois requer o treinamento de múltiplos modelos.

In [70]:
from sklearn.preprocessing import StandardScaler

# Remover a coluna 'date'
X_train = X_train.drop('date', axis=1)
X_test = X_test.drop('date', axis=1)

# Criar um objeto StandardScaler
scaler = StandardScaler()

# Ajustar e transformar os dados de treinamento
X_train_scaled = scaler.fit_transform(X_train)

# Transformar os dados de teste
X_test_scaled = scaler.transform(X_test)

# Verificar os primeiros registros do conjunto de treino padronizado
print(X_train_scaled[:5])

# Verificar os primeiros registros do conjunto de teste padronizado
print(X_test_scaled[:5])

[[ 0.46892995  0.47334124  0.49422781  0.49199214 -0.52823856  0.37693146
   0.74759394 -1.04108068  0.20932832  0.26921029  0.28615145  0.11124338
   0.18306414  1.4903423   0.38791361 -0.23200239  0.21673458  0.38476884]
 [-1.2364618  -1.23998191 -1.22931193 -1.23518481  0.03440603  1.41368975
   1.70213968  1.59341764  0.79482387 -1.2968058  -1.28361321 -1.28815625
  -1.2915147   0.21915177 -1.25066176 -0.93777817 -1.30924984 -1.31366608]
 [ 0.55524377  0.54140748  0.49951635  0.47070995 -0.37531315 -0.62202451
  -0.48480687 -0.30123824 -0.42431643  0.55049135  0.53739912  0.57182654
   0.50937258  0.06489884  0.51279848  0.9906032   0.5556677   0.56833072]
 [ 0.52978595  0.54655143  0.55326161  0.57063368 -0.46403199  0.76787605
   0.98389393  0.3360819   0.38741029  0.32473465  0.35045068  0.1610754
   0.23211429  1.36091813  0.48346253  0.0748104   0.33774994  0.42058551]
 [ 0.00250766 -0.00766928  0.01414821 -0.01211385 -0.46151791 -0.37683087
  -0.89014487  0.57044723  0.299957

# Modelagem e treinamento

**Selecione os algoritmos mais indicados para o problema e dataset escolhidos, justificando as suas escolhas:**

Dado que estamos lidando com um problema de regressão, podemos começar com modelos de regressão simples, como a Regressão Linear. Também podemos experimentar modelos mais complexos, como Árvores de Decisão, Florestas Aleatórias e Máquinas de Vetores de Suporte (SVM). Esses modelos são comumente usados para problemas de regressão e podem ser capazes de capturar padrões complexos nos dados.

In [71]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Criar os modelos
linear_model = LinearRegression()
tree_model = DecisionTreeRegressor(random_state=42)
forest_model = RandomForestRegressor(random_state=42)
svm_model = SVR()

**Há algum ajuste inicial para os hiperparâmetros?:**

Inicialmente, podemos usar os hiperparâmetros padrão fornecidos pelas implementações dos modelos no scikit-learn. No entanto, é provável que precisemos ajustar esses hiperparâmetros para obter o melhor desempenho do modelo.

**O modelo foi devidamente treinado?**

Foi observado problema de underfitting?: Para verificar se o modelo foi devidamente treinado e se há algum problema de underfitting, precisamos treinar o modelo e avaliar seu desempenho. Underfitting ocorre quando o modelo é muito simples para capturar a complexidade dos dados. Se observarmos que o modelo tem um desempenho ruim tanto nos dados de treinamento quanto nos dados de teste, isso pode ser um sinal de underfitting.



In [72]:
# Treinar os modelos
linear_model.fit(X_train_scaled, y_train)
tree_model.fit(X_train_scaled, y_train)
forest_model.fit(X_train_scaled, y_train)
svm_model.fit(X_train_scaled, y_train)



SVR()

In [73]:
from sklearn.metrics import mean_squared_error

# Usar o modelo para fazer previsões nos dados de teste
y_pred = svm_model.predict(X_test_scaled)

# Calcular o MSE
mse = mean_squared_error(y_test, y_pred)

print("Erro Quadrático Médio (MSE) do modelo SVR:", mse)


Erro Quadrático Médio (MSE) do modelo SVR: 3040.142732559128


**Otimizar os hiperparâmetros de algum dos modelos**

Nesta etapa, realizei a otimização de hiperparâmetros para o modelo de Floresta Aleatória. A otimização de hiperparâmetros é um processo importante que pode ajudar a melhorar o desempenho do modelo. Usei o método RandomizedSearchCV do scikit-learn para realizar essa otimização. Esse método realiza uma busca aleatória sobre os hiperparâmetros, o que pode ser mais eficiente do que uma busca exaustiva.

Especifiquei a distribuição dos hiperparâmetros que queria otimizar, que incluía o número de estimadores (n_estimators), a profundidade máxima das árvores (max_depth) e o número mínimo de amostras necessárias para dividir um nó interno (min_samples_split).



In [74]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Definir a distribuição dos hiperparâmetros que queremos otimizar
param_dist = {
    'n_estimators': randint(100, 300),
    'max_depth': [None] + list(randint(1, 20).rvs(size=10)),
    'min_samples_split': randint(2, 10)
}

# Criar o objeto RandomizedSearchCV
random_search = RandomizedSearchCV(forest_model, param_dist, cv=5, scoring='neg_mean_squared_error', n_iter=10, random_state=42)

# Realizar a otimização de hiperparâmetros
random_search.fit(X_train_scaled, y_train)

# Imprimir os melhores hiperparâmetros encontrados
print("Melhores hiperparâmetros:", random_search.best_params_)



Melhores hiperparâmetros: {'max_depth': 7, 'min_samples_split': 8, 'n_estimators': 221}


**Há algum método avançado ou mais complexo que possa ser avaliado?**

Treinando um modelo de Gradient Boosting com o hiperparâmetro random_state definido como 42.



In [75]:
from sklearn.ensemble import GradientBoostingRegressor

# Criar o modelo
gb_model = GradientBoostingRegressor(random_state=42)

# Treinar o modelo
gb_model.fit(X_train_scaled, y_train)


GradientBoostingRegressor(random_state=42)

Agora que o modelo foi treinado, posso usá-lo para fazer previsões nos dados de teste ou avaliar seu desempenho. Por exemplo, posso usar o modelo de Gradient Boosting para fazer previsões e calcular o erro quadrático médio (Mean Squared Error, MSE):

In [76]:
from sklearn.metrics import mean_squared_error

# Usar o modelo para fazer previsões nos dados de teste
y_pred = gb_model.predict(X_test_scaled)

# Calcular o MSE
mse = mean_squared_error(y_test, y_pred)

print("Erro Quadrático Médio (MSE) do modelo de Gradient Boosting:", mse)


Erro Quadrático Médio (MSE) do modelo de Gradient Boosting: 81.72959827538037


**Posso criar um comitê de modelos diferentes para o problema (ensembles)?**

Treinando um ensemble de modelos usando o método de votação com quatro modelos diferentes: Regressão Linear, Árvore de Decisão, Floresta Aleatória e Máquina de Vetores de Suporte para Regressão (SVR).

In [77]:
from sklearn.ensemble import VotingRegressor

# Criar o ensemble
ensemble_model = VotingRegressor([('linear', linear_model), ('tree', tree_model), ('forest', forest_model), ('svm', svm_model)])

# Treinar o ensemble
ensemble_model.fit(X_train_scaled, y_train)


VotingRegressor(estimators=[('linear', LinearRegression()),
                            ('tree', DecisionTreeRegressor(random_state=42)),
                            ('forest', RandomForestRegressor(random_state=42)),
                            ('svm', SVR())])

Usando o ensemble para fazer previsões e calcular o erro quadrático médio (Mean Squared Error, MSE):


In [78]:
from sklearn.metrics import mean_squared_error

# Usar o ensemble para fazer previsões nos dados de teste
y_pred = ensemble_model.predict(X_test_scaled)

# Calcular o MSE
mse = mean_squared_error(y_test, y_pred)

print("Erro Quadrático Médio (MSE) do ensemble de modelos:", mse)

Erro Quadrático Médio (MSE) do ensemble de modelos: 262.9497245620958


# Avaliação de Resultados


**Selecione as métricas de avaliação condizentes com o problema, justificando:**

Para problemas de regressão usarei as métricas mais comuns que incluem o Erro Quadrático Médio (MSE), a Raiz do Erro Quadrático Médio (RMSE) e o Coeficiente de Determinação (R^2). O MSE e o RMSE são úteis porque penalizam erros maiores mais do que erros menores. O R^2 é útil porque fornece uma medida de quanta variância nos dados é explicada pelo modelo.

**Treine o modelo escolhido com toda a base de treino, e teste-o com a base de teste.**

Calculando as métricas de avaliação para cada modelo.

In [79]:
import numpy as np

# Lista de modelos
models = [linear_model, tree_model, forest_model, svm_model, gb_model, ensemble_model]

# Dicionário para armazenar os resultados
results = {}

# Calcular as métricas para cada modelo
for model in models:
    # Fazer previsões
    y_pred = model.predict(X_test_scaled)

    # Calcular as métricas
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Armazenar os resultados
    results[model.__class__.__name__] = {'MSE': mse, 'RMSE': rmse, 'R^2': r2}

# Imprimir os resultados
for model, metrics in results.items():
    print(f"Modelo: {model}")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")
    print("\n")

Modelo: LinearRegression
MSE: 69.80523900031406
RMSE: 8.354952962184411
R^2: 0.9975656145440126


Modelo: DecisionTreeRegressor
MSE: 163.01768807975074
RMSE: 12.767838034677238
R^2: 0.9943149268648983


Modelo: RandomForestRegressor
MSE: 82.51666447982011
RMSE: 9.083868365394784
R^2: 0.9971223167377828


Modelo: SVR
MSE: 3040.142732559128
RMSE: 55.137489356690224
R^2: 0.8939781689991129


Modelo: GradientBoostingRegressor
MSE: 81.72959827538037
RMSE: 9.040442371664142
R^2: 0.9971497648569846


Modelo: VotingRegressor
MSE: 262.9497245620958
RMSE: 16.21572460798764
R^2: 0.9908299005304317




**Os resultados fazem sentido?:**

Sim, os resultados fazem sentido. Todos os modelos foram capazes de fazer previsões com um grau razoável de precisão, como indicado pelos valores de R^2 próximos de 1. No entanto, o modelo SVR teve um desempenho significativamente pior do que os outros modelos, como indicado pelo seu maior MSE e RMSE e menor R^2.

In [80]:
# Fazer previsões nos dados de treinamento
y_train_pred = linear_model.predict(X_train_scaled)

# Calcular as métricas para os dados de treinamento
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

# Fazer previsões nos dados de teste
y_test_pred = linear_model.predict(X_test_scaled)

# Calcular as métricas para os dados de teste
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_test_pred)

# Imprimir as métricas
print(f"Treinamento: MSE = {mse_train}, RMSE = {rmse_train}, R^2 = {r2_train}")
print(f"Teste: MSE = {mse_test}, RMSE = {rmse_test}, R^2 = {r2_test}")


Treinamento: MSE = 73.00701313177987, RMSE = 8.544414147955369, R^2 = 0.9973159325313505
Teste: MSE = 69.80523900031406, RMSE = 8.354952962184411, R^2 = 0.9975656145440126


**Foi observado algum problema de overfitting?**

Com base nos resultados, não parece haver um problema de overfitting com o modelo de Regressão Linear. As métricas de erro (MSE, RMSE) são semelhantes para os conjuntos de treinamento e teste, e a métrica R^2 é alta (próxima de 1) para ambos, o que indica que o modelo está explicando uma grande proporção da variância nos dados.


**Compare os resultados de diferentes modelos.**

Os modelos de Regressão Linear, Floresta Aleatória e Gradient Boosting que eu treinei tiveram desempenhos semelhantes, com valores de MSE e RMSE relativamente baixos e valores de R^2 muito próximos de 1. O modelo de Árvore de Decisão teve um desempenho um pouco pior, e o modelo SVR teve um desempenho significativamente pior. O VotingRegressor, que é um ensemble dos quatro modelos, teve um desempenho intermediário.


**Descreva a melhor solução encontrada, justificando.**

Com base nas métricas de avaliação fornecidas, o modelo de Regressão Linear parece ser a melhor solução, pois tem o menor MSE e RMSE e o maior R^2. No entanto, os modelos de Floresta Aleatória e Gradient Boosting também tiveram desempenhos muito semelhantes. Esses modelos são capazes de capturar relações não lineares e interações entre recursos, o que pode explicar seu bom desempenho. A escolha do melhor modelo também pode depender de outros fatores, como a interpretabilidade do modelo e o tempo de treinamento e previsão. Por exemplo, embora o modelo de Regressão Linear seja mais fácil de interpretar, os modelos de Floresta Aleatória e Gradient Boosting podem ser capazes de capturar padrões mais complexos nos dados.
